In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

In [31]:
data = pd.read_csv('/kaggle/input/gas-prices-in-brazil/2004-2021.tsv',delimiter='\t')

In [32]:
data

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
0,2004-05-09,2004-05-15,CENTRO OESTE,DISTRITO FEDERAL,ETANOL HIDRATADO,127,R$/l,1.288,0.016,1.190,1.350,0.463,0.012,0.825,0.11,0.4201,0.9666,0.133
1,2004-05-09,2004-05-15,CENTRO OESTE,GOIAS,ETANOL HIDRATADO,387,R$/l,1.162,0.114,0.890,1.449,0.399,0.098,0.763,0.088,0.5013,1.05,0.115
2,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO,ETANOL HIDRATADO,192,R$/l,1.389,0.097,1.180,1.760,0.419,0.070,0.97,0.095,0.5614,1.161,0.098
3,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO DO SUL,ETANOL HIDRATADO,162,R$/l,1.262,0.070,1.090,1.509,0.432,0.055,0.83,0.119,0.5991,1.22242,0.143
4,2004-05-09,2004-05-15,NORDESTE,ALAGOAS,ETANOL HIDRATADO,103,R$/l,1.181,0.078,1.050,1.400,0.24,0.066,0.941,0.077,0.7441,1.0317,0.082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120818,2021-04-25,2021-05-01,NORTE,RORAIMA,OLEO DIESEL S10,12,R$/l,4.582,0.056,4.510,4.690,-99999.0,0.012,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
120819,2021-04-25,2021-05-01,SUL,SANTA CATARINA,OLEO DIESEL S10,100,R$/l,4.141,0.187,3.838,4.739,-99999.0,0.045,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
120820,2021-04-25,2021-05-01,SUDESTE,SAO PAULO,OLEO DIESEL S10,886,R$/l,4.246,0.216,3.769,5.199,-99999.0,0.051,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0
120821,2021-04-25,2021-05-01,NORDESTE,SERGIPE,OLEO DIESEL S10,33,R$/l,4.464,0.205,4.190,4.699,-99999.0,0.046,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0


In [33]:
data = data.sample(frac=0.5)

In [34]:
nan_cols = data.columns[data.isnull().sum() > 0]

In [35]:
{col : data[col].unique() for col in data.columns if data[col].dtypes == 'object'}

{'DATA INICIAL': array(['2021-02-21', '2016-06-19', '2009-01-25', '2010-09-19',
        '2012-09-30', '2010-10-10', '2006-09-03', '2020-05-10',
        '2014-10-05', '2014-01-12', '2005-01-16', '2014-09-07',
        '2012-12-02', '2019-12-22', '2019-08-04', '2017-12-10',
        '2006-01-01', '2008-05-25', '2018-12-16', '2014-05-11',
        '2016-07-03', '2021-01-31', '2009-04-19', '2004-12-05',
        '2017-11-05', '2006-09-17', '2016-09-04', '2010-12-26',
        '2011-02-27', '2012-12-30', '2015-03-01', '2015-08-23',
        '2020-04-12', '2017-04-02', '2014-04-20', '2014-09-14',
        '2016-10-09', '2016-12-18', '2008-12-21', '2006-04-09',
        '2017-01-08', '2013-06-23', '2008-06-15', '2014-05-04',
        '2006-07-23', '2020-08-09', '2014-08-17', '2009-03-15',
        '2009-06-14', '2017-04-30', '2012-07-29', '2020-01-19',
        '2010-11-14', '2007-03-11', '2007-05-27', '2009-04-05',
        '2006-05-28', '2013-03-31', '2017-12-03', '2017-02-19',
        '2019-05-12', '2

In [36]:
data['YEAR'] = data['DATA INICIAL'].apply(lambda x : x[0:4])
data['Month'] = data['DATA INICIAL'].apply(lambda x : x[5:7])
data = data.drop(['DATA FINAL','DATA INICIAL'],axis=1)

In [37]:
nominal_fets = ['REGIÃO','ESTADO','UNIDADE DE MEDIDA','MARGEM MÉDIA REVENDA']
nominal_prefix = ['P_','E_','U_','MMR_']

for col , prefix in zip(nominal_fets,nominal_prefix):
    dummie = pd.get_dummies(data[col],prefix=prefix)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(col,axis=1)

In [38]:
data = data.replace('-',np.NaN)
for col in nan_cols:
    data[col] = data[col].fillna(data[col].mean())

In [39]:
encoder = LabelEncoder()
data['PRODUTO'] = encoder.fit_transform(data['PRODUTO'])

In [40]:
y = data['PRODUTO']
x = data.drop('PRODUTO',axis=1)

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=7)

In [ ]:
weights = dict(
    enumerate(
        compute_class_weight(
            'balanced',
            classes = y.unique(),
            y = y
        )
    )  
)

In [ ]:
weights

In [ ]:
x.shape

In [ ]:
len(y.unique())

In [ ]:
inputs = tf.keras.Input(shape=(8728,))
x = tf.keras.layers.Dense(256,activation='relu')(inputs)
x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
outputs = tf.keras.layers.Dense(9,activation='softmax')(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
with tf.device('GPU:0'):
    hist = model.fit(
        x_train,
        y_train,
        validation_data=(x_test,y_test),
        validation_split=0.2,
        class_weight=weights,
        epochs = 100,
        batch_size = 40,
        callbacks=tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            restore_best_weights=True,
            patience=3
        )

    )
